In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib as plt

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
pd.read_csv("../raw_data/facturation_train.csv").head(10)

,ID_CPTE,PERIODID_MY,StatementDate,CurrentTotalBalance,CashBalance,CreditLimit,DelqCycle
0,99690111,2015-05-01,2015-05-03,8497.84,4293.12,16200.0,0
1,99690111,2014-11-01,2014-11-03,866.00,0.00,12000.0,0
2,99690111,2015-06-01,2015-05-31,10790.95,5224.44,16200.0,0
3,99690111,2015-10-01,2015-10-04,12388.46,4786.08,16200.0,0
4,99690111,2015-11-01,2015-11-02,12746.50,4818.48,16200.0,0
5,99690111,2015-08-01,2015-08-02,10610.05,4753.35,16200.0,0
6,99690111,2015-09-01,2015-08-30,11417.12,4616.46,16200.0,0
7,99690111,2014-12-01,2014-12-03,1151.85,0.00,12000.0,0
8,99690111,2015-02-01,2015-01-31,4045.67,1148.45,16200.0,0
9,99690111,2015-12-01,2015-11-30,13119.60,4791.44,16200.0,0


In [3]:
performance_train = pd.read_csv("../raw_data/performance_train.csv")
performance_train.head(10)

,ID_CPTE,PERIODID_MY,Default
0,99690111,2015-12-01,0
1,57427180,2012-12-01,0
2,29617912,2015-12-01,0
3,61632809,2015-12-01,0
4,14117855,2013-12-01,0
5,23700394,2013-12-01,0
6,27881705,2012-12-01,0
7,46100731,2012-12-01,0
8,58512689,2014-12-01,0
9,24661392,2016-12-01,0


In [4]:
paiements_train = pd.read_csv("../raw_data/paiements_train.csv")
paiements_train.head(10)

,ID_CPTE,TRANSACTION_AMT,TRANSACTION_DTTM,PAYMENT_REVERSAL_XFLG
0,99690111,208.0,2015-04-26 00:00:00,Q
1,99690111,176.8,2015-05-28 00:00:00,Q
2,99690111,200.0,2015-03-27 04:00:00,Q
3,99690111,80.8,2015-04-02 00:00:00,Q
4,99690111,250.0,2015-11-24 00:00:00,Q
5,99690111,273.0,2015-12-26 00:00:00,Q
6,99690111,267.5,2015-08-23 00:00:00,Q
7,99690111,618.0,2015-07-27 00:00:00,Q
8,99690111,226.6,2015-09-30 00:00:00,Q
9,99690111,244.8,2015-10-27 00:00:00,Q


In [5]:
transactions_train = pd.read_csv("../raw_data/transactions_train.csv")
transactions_train.head(10)

,ID_CPTE,MERCHANT_CATEGORY_XCD,MERCHANT_CITY_NAME,MERCHANT_COUNTRY_XCD,DECISION_XCD,PRIOR_CREDIT_LIMIT_AMT,TRANSACTION_AMT,TRANSACTION_CATEGORY_XCD,TRANSACTION_DTTM,TRANSACTION_TYPE_XCD,SICGROUP
0,99690111,A,365767,DP,C,5927.0,52.53,E,2015-06-20 12:00:00,F,AN
1,99690111,L,2635650,DP,C,13343.0,28.35,B,2015-01-25 12:00:00,F,AN
2,99690111,L,2635650,DP,C,13343.0,0.00,A,2015-01-26 12:00:00,G,AN
3,99690111,J,680536,AF,C,9430.0,0.00,A,2015-03-25 08:00:00,G,AW
4,99690111,J,680536,AF,C,10600.0,0.00,A,2015-03-03 08:00:00,G,AW
5,99690111,J,680536,AF,C,12203.0,7.28,C,2015-02-16 16:00:00,B,AW
6,99690111,J,680536,AF,C,13314.0,7.00,C,2015-01-20 12:00:00,B,AW
7,99690111,J,680536,AF,C,9696.0,7.14,C,2015-01-11 08:00:00,B,AW
8,99690111,J,680536,AF,C,12203.0,0.00,A,2015-02-20 16:00:00,G,AW
9,99690111,J,680536,AF,C,10600.0,6.36,C,2015-03-04 08:00:00,B,AW


In [71]:
FeatureDataSet = pd.read_csv("../intermidiate_data/20180709_2_features.csv")
X_test = pd.read_csv("../intermidiate_data/20180709_2_features_test.csv")

In [7]:
Input_y = FeatureDataSet["default"]
Input_X = FeatureDataSet.drop(columns = ["default"])

In [8]:
X_train, X_validation, y_train, y_validation = train_test_split(Input_X, Input_y, test_size = 0.2, random_state = 20)

In [12]:
estimator = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1',n_estimators = 20, num_leaves = 38)

param_grid = {'n_estimators': [x for x in range(10, 40, 3)],
              'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
gridsearch = GridSearchCV(estimator, param_grid)

gridsearch.fit(X_train, y_train,
               eval_set=[(X_validation, y_validation)],
               eval_metric=['auc', 'binary_logloss'],
               early_stopping_rounds=5)

[1]	valid_0's binary_logloss: 0.469647	valid_0's auc: 0.837907
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.450215	valid_0's auc: 0.844325
[3]	valid_0's binary_logloss: 0.433524	valid_0's auc: 0.846203
[4]	valid_0's binary_logloss: 0.42099	valid_0's auc: 0.848016
[5]	valid_0's binary_logloss: 0.410131	valid_0's auc: 0.851509
[6]	valid_0's binary_logloss: 0.400447	valid_0's auc: 0.854392
[7]	valid_0's binary_logloss: 0.392604	valid_0's auc: 0.856369
[8]	valid_0's binary_logloss: 0.385344	valid_0's auc: 0.858905
[9]	valid_0's binary_logloss: 0.379321	valid_0's auc: 0.860357
[10]	valid_0's binary_logloss: 0.373554	valid_0's auc: 0.862528
Did not meet early stopping. Best iteration is:
[10]	valid_0's binary_logloss: 0.373554	valid_0's auc: 0.862528


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[1]	valid_0's binary_logloss: 0.471715	valid_0's auc: 0.823479
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.452378	valid_0's auc: 0.83197
[3]	valid_0's binary_logloss: 0.436167	valid_0's auc: 0.841463
[4]	valid_0's binary_logloss: 0.421961	valid_0's auc: 0.84586
[5]	valid_0's binary_logloss: 0.410657	valid_0's auc: 0.84882
[6]	valid_0's binary_logloss: 0.402318	valid_0's auc: 0.850205
[7]	valid_0's binary_logloss: 0.39476	valid_0's auc: 0.851931
[8]	valid_0's binary_logloss: 0.387508	valid_0's auc: 0.853874
[9]	valid_0's binary_logloss: 0.381297	valid_0's auc: 0.855599
[10]	valid_0's binary_logloss: 0.376088	valid_0's auc: 0.857508
Did not meet early stopping. Best iteration is:
[10]	valid_0's binary_logloss: 0.376088	valid_0's auc: 0.857508


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[1]	valid_0's binary_logloss: 0.470735	valid_0's auc: 0.823169
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.450915	valid_0's auc: 0.844272
[3]	valid_0's binary_logloss: 0.434549	valid_0's auc: 0.849207
[4]	valid_0's binary_logloss: 0.420812	valid_0's auc: 0.852927
[5]	valid_0's binary_logloss: 0.410605	valid_0's auc: 0.851846
[6]	valid_0's binary_logloss: 0.402066	valid_0's auc: 0.853158
[7]	valid_0's binary_logloss: 0.394556	valid_0's auc: 0.852809
[8]	valid_0's binary_logloss: 0.387735	valid_0's auc: 0.854413
[9]	valid_0's binary_logloss: 0.381917	valid_0's auc: 0.854194
[10]	valid_0's binary_logloss: 0.376924	valid_0's auc: 0.856206
Did not meet early stopping. Best iteration is:
[10]	valid_0's binary_logloss: 0.376924	valid_0's auc: 0.856206


...

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[5]	valid_0's binary_logloss: 0.375007	valid_0's auc: 0.855335
[6]	valid_0's binary_logloss: 0.369813	valid_0's auc: 0.855791
[7]	valid_0's binary_logloss: 0.364555	valid_0's auc: 0.856613
[8]	valid_0's binary_logloss: 0.360715	valid_0's auc: 0.858285
[9]	valid_0's binary_logloss: 0.356826	valid_0's auc: 0.859615
[10]	valid_0's binary_logloss: 0.354304	valid_0's auc: 0.860612
[11]	valid_0's binary_logloss: 0.352709	valid_0's auc: 0.860332
[12]	valid_0's binary_logloss: 0.351001	valid_0's auc: 0.860839
[13]	valid_0's binary_logloss: 0.35001	valid_0's auc: 0.861
[14]	valid_0's binary_logloss: 0.350839	valid_0's auc: 0.860171
[15]	valid_0's binary_logloss: 0.350144	valid_0's auc: 0.860424
[16]	valid_0's binary_logloss: 0.350309	valid_0's auc: 0.859944
[17]	valid_0's binary_logloss: 0.349668	valid_0's auc: 0.861196
[18]	valid_0's binary_logloss: 0.349975	valid_0's auc: 0.86114
[19]	valid_0's binary_logloss: 0.349779	valid_0's auc: 0.861429
[20]	valid_0's binary_logloss: 0.351471	valid_0's 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[5]	valid_0's binary_logloss: 0.397486	valid_0's auc: 0.856817
[6]	valid_0's binary_logloss: 0.388705	valid_0's auc: 0.856956
[7]	valid_0's binary_logloss: 0.379977	valid_0's auc: 0.859821
[8]	valid_0's binary_logloss: 0.373975	valid_0's auc: 0.86054
[9]	valid_0's binary_logloss: 0.368672	valid_0's auc: 0.861986
[10]	valid_0's binary_logloss: 0.364785	valid_0's auc: 0.862685
[11]	valid_0's binary_logloss: 0.360898	valid_0's auc: 0.863273
[12]	valid_0's binary_logloss: 0.358209	valid_0's auc: 0.862506
[13]	valid_0's binary_logloss: 0.354968	valid_0's auc: 0.864069
[14]	valid_0's binary_logloss: 0.352916	valid_0's auc: 0.864478
[15]	valid_0's binary_logloss: 0.351246	valid_0's auc: 0.864372
[16]	valid_0's binary_logloss: 0.350116	valid_0's auc: 0.863962
[17]	valid_0's binary_logloss: 0.348922	valid_0's auc: 0.863989
[18]	valid_0's binary_logloss: 0.346999	valid_0's auc: 0.86502
[19]	valid_0's binary_logloss: 0.345504	valid_0's auc: 0.865755
Did not meet early stopping. Best iteration is:

GridSearchCV(cv=None, error_score='raise',
       estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.125, max_depth=-1, metric='l1',
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=20, n_jobs=-1, num_leaves=38, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 13, 16, 19, 22, 25, 28, 31, 34, 37], 'learning_rate': [0.1, 0.125, 0.15, 0.175, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [13]:
gridsearch.best_params_

{'learning_rate': 0.125, 'n_estimators': 19}

In [14]:
gbm = lgb.LGBMClassifier(learning_rate = 0.125, metric = "11", n_estumaters = 19, num_leaves = 38)
gbm.fit(X_train, y_train,
        eval_set=[(X_validation, y_validation)],
        eval_metric=['auc', 'binary_logloss'],
        early_stopping_rounds=5)

[1]	valid_0's binary_logloss: 0.464118	valid_0's auc: 0.839068
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: 0.440587	valid_0's auc: 0.843944
[3]	valid_0's binary_logloss: 0.421871	valid_0's auc: 0.851053
[4]	valid_0's binary_logloss: 0.408327	valid_0's auc: 0.854831
[5]	valid_0's binary_logloss: 0.397486	valid_0's auc: 0.856817
[6]	valid_0's binary_logloss: 0.388705	valid_0's auc: 0.856956
[7]	valid_0's binary_logloss: 0.379977	valid_0's auc: 0.859821
[8]	valid_0's binary_logloss: 0.373975	valid_0's auc: 0.86054
[9]	valid_0's binary_logloss: 0.368672	valid_0's auc: 0.861986
[10]	valid_0's binary_logloss: 0.364785	valid_0's auc: 0.862685
[11]	valid_0's binary_logloss: 0.360898	valid_0's auc: 0.863273
[12]	valid_0's binary_logloss: 0.358209	valid_0's auc: 0.862506
[13]	valid_0's binary_logloss: 0.354968	valid_0's auc: 0.864069
[14]	valid_0's binary_logloss: 0.352916	valid_0's auc: 0.864478
[15]	valid_0's binary_logloss: 0.351246	valid_0's auc

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.125, max_depth=-1, metric='11',
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_estumaters=19, n_jobs=-1, num_leaves=38,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [45]:
result = gbm.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [72]:
X_test.shape

(5100, 34)

In [76]:
Result = X_test["id_cpte"].to_frame()
Result["default"] = result

In [78]:
Result.reset

In [82]:
Result.to_csv("../result/0711_lightgbm_result.csv", index = False)

5100